# A simple example
In this notebook, we show you how to model and solve a simple flexible job shop problem with PyJobShop.

A scheduling problem consists of three main elements:

- **Machines**: a machine is a resource that can process operations.
- **Operations**: an operation is a task that needs to be scheduled, i.e., its starting time, ending time and selected machine must be decided.
- **Jobs**: a job represents a collection of operations and can be used to measure performance metrics such as completion times or tardiness.

The Flexible Job Shop Problem (FJSP) is a commonly studied scheduling problem that generalizes many known scheduling problem variants. 
In the FJSP, there is a set of machines $M$ and a set of jobs $J$.
Each job is composed of a sequence of operations denoted by $O$, which must be performed in sequence.
Each operation needs to be processed by exactly one machine that is selected from a set of eligible machines.
The main goal of the FJSP is commonly to minimize the makespan.

Let's consider a simple example from [Google OR-Tools](https://github.com/google/or-tools/blob/master/examples/python/flexible_job_shop_sat.py). Below we have a data instance with three machines and three jobs, each job consisting of three operations.

In [ ]:
NUM_MACHINES = 3

# Each job consists of a list of operations. An operation is represented
# by a list of tuples (processing_time, machine), denoting the eligible
# machine assignments and corresponding processing times.
data = [
    [  # Job with three operations
        [(3, 0), (1, 1), (5, 2)],  # Operation with three eligible machines
        [(2, 0), (4, 1), (6, 2)],
        [(2, 0), (3, 1), (1, 2)],
    ],
    [
        [(2, 0), (3, 1), (4, 2)],
        [(1, 0), (5, 1), (4, 2)],
        [(2, 0), (1, 1), (4, 2)],
    ],
    [
        [(2, 0), (1, 1), (4, 2)],
        [(2, 0), (3, 1), (4, 2)],
        [(3, 0), (1, 1), (5, 2)],
    ],
]

PyJobShop provides a simple modeling interface through its `Model` class.
Let's import the `Model` class and initialize it.

In [ ]:
from pyjobshop import Model

m = Model()

Data objects such as machines, jobs and operations can be created with the `Model.add_*` method.

In [ ]:
machines = [
    m.add_machine(name=f"Machine {idx}") for idx in range(NUM_MACHINES)
]

In [ ]:
jobs = {}
operations = {}

for job_idx, job_data in enumerate(data):
    job = m.add_job(name=f"Job {job_idx}")
    jobs[job_idx] = job

    for idx in range(len(job_data)):
        op_idx = idx = (job_idx, idx)
        operations[op_idx] = m.add_operation(job, name=f"Op {op_idx}")

There are three more things that we need to add to the model:
- Operations have to be associated with their corresponding job;
- Processing times of specific operation and machine combinations must be set;
- Operations of the same job must be processed in a given order;


Let's add the remaining three steps to the model.

In [ ]:
for job_idx, job_data in enumerate(data):
    for idx, operation_data in enumerate(job_data):
        operation = operations[(job_idx, idx)]

        for duration, machine_idx in operation_data:
            machine = machines[machine_idx]
            m.add_processing_time(machine, operation, duration)

    for idx in range(len(job_data) - 1):
        first = operations[(job_idx, idx)]
        second = operations[(job_idx, idx + 1)]
        m.add_timing_precedence(first, second)

Now that we have our model setup correctly, we can solve the model.

In [ ]:
result = m.solve()

The output above is from CP Optimizer and shows the solver output. Turns out we have found the optimal solution! Let's plot that solution.

In [ ]:
from pyjobshop import plot

In [ ]:
data = m.data()
plot(data, result.solution, plot_labels=True)

The plot shows a Gantt chart of our solution. Each row represents a machine and each horizontal bar represents an operation.
The colors of the operations depict the individual jobs they are associated with, with each job having a unique color.

## Summary
This concludes the notebook. We showed how to set up a simple FJSP problem instance using PyJobShop's modeling interface. After setup, we solved the model and plotted the optimal solution.